In [ ]:
%sh git clone https://github.com/ammaryasirnaich/apache_spark_stream_lab.git

In [ ]:
%sh ls apache_spark_stream_lab

In [ ]:
import os
import socket
import os
from _thread import *
import time


In [ ]:
def main():
    ServerSideSocket = socket.socket()
    dataset_path= os.getcwd()+"/apache_spark_stream_lab/nasaAccessLogs.csv"
    file_exists = os.path.exists(dataset_path)
    if (file_exists==False):
      print("Dataset path Error")
      return
    else:    
      print("Dataset path accessible:"+str(file_exists))
      
    host = '127.0.0.1'
    port = 5551
    ThreadCount = 0
    try:
        ServerSideSocket.bind((host, port))
    except socket.error as e:
        print(str(e))
    print('Socket is listening..')
    ServerSideSocket.listen(5)

    def multi_threaded_client(connection):
        connection.send(str.encode('Server is working:')) 
        file_handle = open(dataset_path, 'r')
       
        with open(dataset_path, 'r') as file_handle:
            while True:
                try:
                    line = file_handle.readline()
                except: 
                    connection.close()
                    file_handle.close()   
                
                if line != "":  
                    try:
                        print(line, end='')
                        connection.sendall(line.encode())      
                    except socket.error:
                        connection.close()
                        file_handle.close()
                else:
                    file_handle.seek(0)
                time.sleep(1)

            connection.close()
    while True:
        Client, address = ServerSideSocket.accept()
        print('Connected to: ' + address[0] + ':' + str(address[1]))
        start_new_thread(multi_threaded_client, (Client, ))
        ThreadCount += 1
        print('Thread Number: ' + str(ThreadCount))
    ServerSideSocket.close()


In [ ]:
#start the server
main()

In [ ]:
%sh fuser -k 5551/tcp